In [ ]:
# To convert from NFA to DFA using subset construction method
# 1. Epsilon closure of the initial state
# 2. For each state in the epsilon closure, find the transition on each input symbol
# 3. Epsilon closure of the states reached in step 2
# 4. Repeat step 2 and 3 until no new states are reached
# 5. The states reached in step 4 are the states of the DFA
# 6. The transition function of the DFA is the transitions found in step 2 and 3

In [356]:
def Split_range(range_string):
    start, end = range_string.split('-')
    list_range = []
    try :
        start = int(start)
        end = int(end)
        for i in range(start, end + 1):
            list_range.append(i)
        return list_range
    except ValueError:
        for i in range(ord(start), ord(end) + 1):
            list_range.append(chr(i))
        return list_range

range_string = 'a-z'
Symbols = Split_range(range_string)
print(Symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [357]:
Symbols = ['a', 'b', 'c']
accept_states = {4}
start_state = {0}

In [358]:
def epsilon_closure(nfa, states):
    closure = set(states)
    stack = list(states)
    while stack:
        state = stack.pop()
        if 'e' in nfa[state]:
            for s in nfa[state]['e']:
                if s not in closure:
                    closure.add(s)
                    stack.append(s)
    return closure

def move(nfa, states, symbol):
    moves = set()
    for state in states:
        if symbol in nfa[state]:
            moves.update(nfa[state][symbol])
    return moves

nfa = {
    0: {'a': {2} , 'e': {3} , 'b': {4}},
    1: {'a': {3}, 'b': {2}},
    2: {'a': {2} , 'b': {4}},
    3: {'a' : {3} , 'b': {1}},
    4: {'e': {3} , 'b': {1} , 'a': {2}},
}
D_states = epsilon_closure(nfa,start_state) # {0, 1}
D_transitions = {}
def convert_dfa_from_nfa(nfa , D_states , D_transitions) :
    stack = [D_states]
    while stack:
        D_states = stack.pop()
        for symbol in Symbols:
            moves = move(nfa, D_states, symbol)
            moves = epsilon_closure(nfa, moves)
            if moves:
                if (frozenset(D_states), symbol) not in D_transitions:
                    D_transitions[(frozenset(D_states), symbol)] = moves
                    stack.append(moves)
    return D_transitions


D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)

D_states = set(D_transitions.keys())
set_of_states = set()
dict_of_states = {}
for D_state in D_states:
    set_of_states.add(D_state[0])
for state in (set_of_states):
    for accept_state in accept_states:
        if accept_state in state:
            dict_of_states[str(set(state))] = True
            break
        else:
            dict_of_states[str(set(state))] = False
print(dict_of_states)
print(D_transitions)
#D_transitions = {(frozenset({0, 1, 2}), 'a'): {2, 3}, (frozenset({0, 1, 2}), 'b'): {3, 4}, (frozenset({3, 4}), 'a'): {2}, (frozenset({3, 4}), 'b'): {1, 2}, (frozenset({1, 2}), 'a'): {3}, (frozenset({1, 2}), 'b'): {3, 4}, (frozenset({3}), 'b'): {1, 2}, (frozenset({2}), 'b'): {3, 4}, (frozenset({2, 3}), 'b'): {1, 2, 3, 4}, (frozenset({1, 2, 3, 4}), 'a'): {2, 3}, (frozenset({1, 2, 3, 4}), 'b'): {1, 2, 3, 4}, (frozenset({3}), 'a'): {3, 4}, (frozenset({2}), 'a'): {1, 2, 3, 4}, (frozenset({2, 3}), 'a'): {0, 1, 2}}

{'{2, 3, 4}': True, '{2, 3}': False, '{1, 2}': False, '{0, 3}': False, '{1, 3, 4}': True}
{(frozenset({0, 3}), 'a'): {2, 3}, (frozenset({0, 3}), 'b'): {1, 3, 4}, (frozenset({1, 3, 4}), 'a'): {2, 3}, (frozenset({1, 3, 4}), 'b'): {1, 2}, (frozenset({1, 2}), 'a'): {2, 3}, (frozenset({1, 2}), 'b'): {2, 3, 4}, (frozenset({2, 3, 4}), 'a'): {2, 3}, (frozenset({2, 3, 4}), 'b'): {1, 3, 4}, (frozenset({2, 3}), 'a'): {2, 3}, (frozenset({2, 3}), 'b'): {1, 3, 4}}


In [359]:
from graphviz import Digraph
def visualize_dfa(D_transitions):
    dot = Digraph()

    # Add nodes
    for key in dict_of_states.keys():
        dot.node(key, shape='doublecircle' if dict_of_states[key] else 'circle')

    # Add edges
    for (src, symbol), dst in D_transitions.items():
        dot.edge(str(set(src)), str(set(dst)), label=symbol)

    return dot


#D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)
dot = visualize_dfa(D_transitions)
dot.format = 'png'
dot.render('dfa_graph')

'dfa_graph.png'

In [360]:
new_D_transitions = {}

# Iterate through the original dictionary
for key, value in D_transitions.items():
    # Extract state and symbol from the key
    state = ', '.join(str(s) for s in key[0])
    symbol = key[1]
    # Construct new key in the desired format
    new_key = f'{{{state}}}', symbol
    new_D_transitions[new_key] = value

print(new_D_transitions)
D_transitions = new_D_transitions



{('{0, 3}', 'a'): {2, 3}, ('{0, 3}', 'b'): {1, 3, 4}, ('{1, 3, 4}', 'a'): {2, 3}, ('{1, 3, 4}', 'b'): {1, 2}, ('{1, 2}', 'a'): {2, 3}, ('{1, 2}', 'b'): {2, 3, 4}, ('{2, 3, 4}', 'a'): {2, 3}, ('{2, 3, 4}', 'b'): {1, 3, 4}, ('{2, 3}', 'a'): {2, 3}, ('{2, 3}', 'b'): {1, 3, 4}}


In [342]:
# first_set = set()
# second_set = set()
# for key in dict_of_states.keys():
#     if dict_of_states[key]:
#         second_set.add(key)
#     else:
#         first_set.add(key)
# stack = [first_set, second_set]
# def DFA_Minimization(D_transitions, Symbols, dict_of_states):
#     P = [first_set, second_set]
#     W = [second_set]  
    
#     while W:
#         A = W.pop()
#         for c in Symbols:
#             X = set()
#             for state in dict_of_states.keys():
#                 if (state, c) in D_transitions:
#                     if str(D_transitions[(state, c)]) in A:
#                         X.add(state)
#             for Y in P[:]:
#                 intersection = X & Y
#                 difference = Y - X
#                 if intersection:
#                     P.remove(Y)
#                     P.extend([intersection, difference])
#                     if Y in W:
#                         W.remove(Y)
#                         W.extend([intersection, difference])
#                     else:
#                         if len(intersection) <= len(difference):
#                             W.append(intersection)
#                         else:
#                             W.append(difference)
#     for sets in P[:]:
#         if sets == set():
#             P.remove(sets)  
#     return P

# P = DFA_Minimization(D_transitions, Symbols, dict_of_states)
# li = list(dict_of_states.keys())
# node_mapper = {}
# for i in range(len(P)):
#     node_mapper[f'S{i}'] = (P[i])

# converted_dict = {}
# for key, value_set in node_mapper.items():
#     for value in value_set:
#         converted_dict[value] = key


# minimized_transitions = {}
# for (src, symbol), dst in D_transitions.items():
#     for i in range(len(P)):
#         if src in P[i]:
#             src = f'S{i}'
#             break
#     for i in range(len(P)):
#         if dst in P[i]:
#             dst = f'S{i}'
#             break
#     minimized_transitions[(src, symbol)] = converted_dict[str(dst)]
# print('Minimized Transitions: ',minimized_transitions)
# print("Node mapper : ",node_mapper)
# dict_of_min_states = {}
# for key,sets in node_mapper.items():
#     dict_of_min_states[key] = False
#     for set_ in sets:
#         if dict_of_states[set_]:
#             dict_of_min_states[key] = True
#             break
    
# print((dict_of_min_states))   
# print(P)    

Minimized Transitions:  {('S2', 'a'): 'S2', ('S2', 'b'): 'S3', ('S3', 'a'): 'S2', ('S3', 'b'): 'S1', ('S1', 'a'): 'S2', ('S1', 'b'): 'S0', ('S0', 'a'): 'S2', ('S0', 'b'): 'S3'}
Node mapper :  {'S0': {'{2, 3, 4}'}, 'S1': {'{1, 2}'}, 'S2': {'{2, 3}', '{0, 3}'}, 'S3': {'{1, 3, 4}'}}
{'S0': True, 'S1': False, 'S2': False, 'S3': True}
[{'{2, 3, 4}'}, {'{1, 2}'}, {'{2, 3}', '{0, 3}'}, {'{1, 3, 4}'}]


In [362]:
def minimize_dfa(D_transitions, dict_of_states):
    partitions = [set(), set()]
    for state, is_accepting in dict_of_states.items():
        partitions[is_accepting].add(state)     # Non accepting states then accepting states
    while True:
        new_partitions = []
        for partition in partitions:
            if len(partition) <= 1: 
                new_partitions.append(partition)
                continue

            transition_partitions = {}
            for state in partition:
                transitions = [(symbol, frozenset(D_transitions[state, symbol]) if ((state, symbol) in D_transitions.keys()) else None) for symbol in Symbols] 
                transition_partitions[state] = transitions
            grouped_transitions = {}
            for state, transitions in transition_partitions.items():
                if tuple(transitions) not in grouped_transitions.keys():
                    grouped_transitions[tuple(transitions)] = set()
                grouped_transitions[tuple(transitions)].add(state)

            new_partitions.extend(grouped_transitions.values())

        if new_partitions == partitions: 
            break
        partitions = new_partitions

    # Merge equivalent states
    state_mapping = {}
    for i, partition in enumerate(partitions):
        for state in partition:
            state_mapping[state] = f'S{i}'

    return partitions , state_mapping

partitions , state_mapping = minimize_dfa(D_transitions, dict_of_states)
print("States : " , partitions)
print("State_mapping: " , state_mapping)
node_mapper = {}
for key, value in state_mapping.items():
    if value not in node_mapper:
        node_mapper[value] = set()
    node_mapper[value].add(key)
print("Node mapper : " , node_mapper)
dict_of_min_states = {}
for key,sets in node_mapper.items():
    dict_of_min_states[key] = False
    for set_ in sets:
        if dict_of_states[set_]:
            dict_of_min_states[key] = True
            break
    
print((dict_of_min_states))




minimized_transitions = {}
for (src, symbol), dst in D_transitions.items():
    for i in range(len(partitions)):
        if src in partitions[i]:
            src = f'S{i}'
            break
    for i in range(len(partitions)):
        if dst in partitions[i]:
            dst = f'S{i}'
            break
    minimized_transitions[(src, symbol)] = state_mapping[str(dst)]
print('Minimized Transitions: ',minimized_transitions)


States :  [{'{2, 3}', '{0, 3}'}, {'{1, 2}'}, {'{2, 3, 4}'}, {'{1, 3, 4}'}]
State_mapping:  {'{2, 3}': 'S0', '{0, 3}': 'S0', '{1, 2}': 'S1', '{2, 3, 4}': 'S2', '{1, 3, 4}': 'S3'}
Node mapper :  {'S0': {'{2, 3}', '{0, 3}'}, 'S1': {'{1, 2}'}, 'S2': {'{2, 3, 4}'}, 'S3': {'{1, 3, 4}'}}
{'S0': False, 'S1': False, 'S2': True, 'S3': True}
Minimized Transitions:  {('S0', 'a'): 'S0', ('S0', 'b'): 'S3', ('S3', 'a'): 'S0', ('S3', 'b'): 'S1', ('S1', 'a'): 'S0', ('S1', 'b'): 'S2', ('S2', 'a'): 'S0', ('S2', 'b'): 'S3'}


In [363]:
def visualize_min_dfa(D_transitions , dict_of_min_states):
    dot = Digraph()

    # Add nodes
    for key in dict_of_min_states.keys():
        dot.node(key, shape='doublecircle' if dict_of_min_states[key] else 'circle')
        
    # Add edges
    for (src, symbol), dst in D_transitions.items():
        dot.edge((src), (dst), label=symbol)

    return dot

dot = visualize_min_dfa(minimized_transitions , dict_of_min_states)
dot.format = 'png'
dot.render('dfa_min_graph')

'dfa_min_graph.png'